In [1]:
import sys
sys.path.append('../..')

In [2]:
from skactiveml.stream import RandomSampler, PeriodicSampler
from skactiveml.stream.budget_manager import FixedBudget

In [3]:
RandomSampler(budget_manager=FixedBudget(0.5), random_state=1).query([[1],[1],[1],[1],[1]], return_utilities=True)

NameError: name 'adapt_query_strategy' is not defined

In [4]:
PeriodicSampler(budget_manager=FixedBudget(0.5), random_state=1).query([[1],[1],[1],[1],[1]], return_utilities=True)

NameError: name 'adapt_query_strategy' is not defined